# DO BASIS CHANGE

This file demonstrates how to convert the bunch of the files (overlaps, transition density matrices, and vibronic Hamiltonians in the KS basis) generated in the step2 to vibronic Hamiltonian files in the basis of configuration interaction (CI) functions composed of Slater Determinants (SD)

To start, lets import the required modules

In [1]:
import os
import sys

# Fisrt, we add the location of the library to test to the PYTHON path
if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *
    

from libra_py import hpc_utils
from libra_py import data_read
from libra_py import data_outs
from libra_py import units
from libra_py import QE_methods
from libra_py.workflows.nbra import step3
#import libra_py.workflows.nbra.step2_analysis as step2a

#import numpy as np
#from matplotlib.mlab import griddata

import matplotlib.pyplot as plt   # plots
%matplotlib inline 

plt.rc('axes', titlesize=24)      # fontsize of the axes title
plt.rc('axes', labelsize=20)      # fontsize of the x and y labels
plt.rc('legend', fontsize=20)     # legend fontsize
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels

plt.rc('figure.subplot', left=0.2)
plt.rc('figure.subplot', right=0.95)
plt.rc('figure.subplot', bottom=0.13)
plt.rc('figure.subplot', top=0.88)

colors = {}

colors.update({"11": "#8b1a0e"})  # red       
colors.update({"12": "#FF4500"})  # orangered 
colors.update({"13": "#B22222"})  # firebrick 
colors.update({"14": "#DC143C"})  # crimson   

colors.update({"21": "#5e9c36"})  # green
colors.update({"22": "#006400"})  # darkgreen  
colors.update({"23": "#228B22"})  # forestgreen
colors.update({"24": "#808000"})  # olive      

colors.update({"31": "#8A2BE2"})  # blueviolet
colors.update({"32": "#00008B"})  # darkblue  

colors.update({"41": "#2F4F4F"})  # darkslategray

clrs_index = ["11", "21", "31", "41", "12", "22", "32", "13","23", "14", "24"]

/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/miniconda2/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, s

Lets read in the files

In my case, the files contain 40 x 40 matrices, which are composed of 20 x 20 blocks of alpha and beta orbitals.

In this set of spin-orbitals, the orbital with index 28 is the alpha-HOMO. 

Assume we only need HOMO-1, HOMO, LUMO and LUMO+1 alpha-spin orbitals, so we can define this by setting the "active_space" parameter to the list \[27, 28, 29, 30\]


In [2]:
params = { "data_set_paths" : [os.getcwd()+"/traj1_in/"],
           "data_dim":40, "active_space":[27, 28, 29, 30],
           "isnap":0,  "fsnap":10,         
           "data_re_prefix" : "S_dia_ks_", "data_re_suffix" : "_re",
           "data_im_prefix" : "S_dia_ks_", "data_im_suffix" : "_im"  }
S = data_read.get_data_sets(params)

params.update({ "data_re_prefix" : "St_dia_ks_", "data_re_suffix" : "_re",
                "data_im_prefix" : "St_dia_ks_", "data_im_suffix" : "_im"  } ) 
St = data_read.get_data_sets(params)

params.update({ "data_re_prefix" : "hvib_dia_", "data_re_suffix" : "_re",
                "data_im_prefix" : "hvib_dia_", "data_im_suffix" : "_im"  } ) 
Hvib_ks = data_read.get_data_sets(params)

Prepare the folder for the final results

In [3]:
# Remove the previous results and temporary working directory from the previous runs
os.system("rm -r traj1_out")

# Create the new results directory
os.system("mkdir traj1_out")

0

Setup the parameters for the SD construction and for the construction of their superpositions

In the above definition, we have included only HOMO-1, HOMO, LUMO and LUMO+1 orbitals in the calculations. They shall have 4 electrons on them, so the entries defining the SD basis will all contain 4 numbers. Note that the lowest number is 1, not 0.

We will include only 3 SD states: a) GS (just 2 orbitals are occupied), b) HOMO -> LUMO, and c) HOMO -> LUMO+1 transitions with the beta electrons.

We will not add any enery corrections to the resulting SDs.

Also, here we'll use the simplest CIs - those that contain only one of each defined SDs (the Pyxaid-style states)

In [4]:
params = { "SD_basis" : [ [1, -1, 2, -2], [1, -1, 2, -3], [1, -1, 2, -4] ],
           "SD_energy_corr" : [0.0, 0.0, 0.0],
           "CI_basis" : [ [1.0, 0.0, 0.0],
                          [0.0, 1.0, 0.0],
                          [0.0, 0.0, 1.0]
                        ],
          "output_set_paths" : [os.getcwd()+"/traj1_out/"], 
          "dt" : 1.0*units.fs2au,
          "do_othogonalization":1,
          "do_state_reordering":2,
          "do_phase_correction":1,
          "do_output" : 1,
          "Hvib_re_prefix":"Hvib_",  "Hvib_re_suffix":"_re",
          "Hvib_im_prefix":"Hvib_",  "Hvib_im_suffix":"_im"
         }

Hvib = step3.run(S, St, Hvib_ks, params)